In [1]:
import pandas as pd
import os
import datetime
import numpy as np
base_dir = "/home/tanyx/dataDemo/campus2018/not_date"
# frame = pd.read_csv("/home/tanyx/dataDemo/campus2018/no_holiday_all.csv")

In [2]:
frames = []
for csv in os.listdir(base_dir):
    frames.append(pd.read_csv(os.path.join(base_dir, csv), sep=",", encoding="utf-8"))
frame = pd.concat(frames,ignore_index=True)

In [ ]:
frames[0].head()

In [3]:
frame["datetime"] = frame.apply(lambda x: datetime.datetime.strptime(x["time_stamp"], "%Y-%m-%d %H"), axis=1)
frame["month"] = frame.apply(lambda x: x["datetime"].month, axis=1)
frame["day"] = frame.apply(lambda x: x["datetime"].day, axis=1)
frame["hour"] = frame.apply(lambda x: x["datetime"].hour, axis=1)
frame["week"] = frame.apply(lambda x: x["day"]//7, axis=1)
frame["week_day"] = frame.apply(lambda x: x["datetime"].weekday(), axis=1)

In [4]:
print(frame.shape)

(160385, 9)


In [5]:
# 按照loc_id对应的值来删除数据
class Location(object):

    def __init__(self):
        self.under_dorms = [18, 30, 23, 11, 31, 6, 3, 32, 7, 9, 5, 1, 2, 19]
        self.master_dorms = [17, 28, 21, 25, 4]
        self.dining_halls = [29, 10, 12, 8]
        self.classrooms = [22, 15, 16, 24, 14, 22, 27]
        self.labs = [20]

    def check_loc(self, loc_id):
        if loc_id in self.under_dorms:
            return 0
        if loc_id in self.master_dorms:
            return 1
        if loc_id in self.dining_halls:
            return 2
        if loc_id in self.classrooms:
            return 3
        if loc_id in self.labs:
            return 4
location = Location()

In [6]:
class Day(object):
    def __init__(self):
        self.holidays = [
            datetime.date(2017, 1, 1),
            datetime.date(2017, 1, 2),
            datetime.date(2017, 4, 2),
            datetime.date(2017, 4, 3),
            datetime.date(2017, 4, 4),
            datetime.date(2017, 4, 29),
            datetime.date(2017, 4, 30),
            datetime.date(2017, 5, 1),
            datetime.date(2017, 5, 28),
            datetime.date(2017, 5, 29),
            datetime.date(2017, 5, 30),
            datetime.date(2017, 10, 1),
            datetime.date(2017, 10, 2),
            datetime.date(2017, 10, 3),
            datetime.date(2017, 10, 4),
            datetime.date(2017, 10, 5),
            datetime.date(2017, 10, 6),
            datetime.date(2017, 10, 7),
            datetime.date(2017, 10, 8),
            datetime.date(2017, 12, 30),
            datetime.date(2017, 12, 31)
        ]
        self.winter_vocation = (datetime.date(2017, 1, 21),
                                datetime.date(2017, 2, 25))
        self.summer_vocation = (datetime.date(2017, 7, 15),
                                datetime.date(2017, 8, 29))
        self.still_working = [
            datetime.date(2017, 4, 1),
            datetime.date(2017, 5, 27),
            datetime.date(2017, 9, 30)
        ]

    def check_day(self, day):
        if (self.winter_vocation[0] < day < self.winter_vocation[1]) or (self.summer_vocation[0] < day < self.summer_vocation[1]):
            return 3  # 长假
        if day in self.holidays:
            return 2  # 短假
        if (day.weekday() > 4) and (day not in self.still_working):
            return 1  # 周末
        return 0  # 工作

check_day = Day()

修正点的规则：
    1、教学楼去掉所有假期数据 -- 3
    2、本科生宿舍去掉假期数据 -- 0
    3、研究生宿舍去掉平时假期的数据（7月份的不去）-- 1
    4、食堂去掉所有假期的数据 -- 2
    5、科研楼去掉平时假期的数据 -- 4

In [ ]:
frame.head()

In [7]:
num_nan = 0
n_frames = []
print(len(frame))
for month in [1,3,4,5,6,7,9,10]:
    day_max = 31
    if month in [4, 6, 9]:
        day_max -=1
    print(month, day_max)
    for day in range(1, day_max+1):
        for hour in range(24):
            for loc_id in range(1,34):
                if len(frame[(frame["day"]==day)&(frame["month"]==month)&(frame["hour"]==hour)&(frame["loc_id"]==loc_id)]) == 0:
                    date_time = datetime.datetime(2017,month,day,hour,0,0)
                    time_stamp = date_time.strftime("%Y-%m-%d %H")
                    num_of_people = frame[(frame["week_day"]==date_time.weekday())&(frame["month"]==month)&(frame["hour"]==hour)&(frame["loc_id"]==loc_id)]["num_of_people"].mean()
                    if num_of_people is not np.nan:
                        n_frames.append([loc_id,time_stamp, num_of_people,date_time, month,day, hour, day//7, date_time.weekday()])
                        num_nan+=1

160385
1 31
3 31
4 30
5 31
6 30
7 31
9 30
10 31


In [8]:
n_f = pd.DataFrame(n_frames,columns=frame.columns)
print(n_f.shape)
frame = pd.concat([frame, n_f], ignore_index=True).sort_values(by=["month","day","hour","loc_id"])
print(frame.shape)

(35535, 9)
(195920, 9)


In [9]:
num_nan2 = 0
# 处理假期值
for i in range(len(frame)):
    loc_id = frame.loc[i,"loc_id"]
    loc = location.check_loc(loc_id)
    day_type = check_day.check_day(frame.loc[i,"datetime"].date())
    if day_type == 3: # 长假
        if loc in [1, 2, 4]:
            week_day = frame.loc[i, "week_day"]
            month = frame.loc[i, "month"]
            hour = frame.loc[i, "hour"]
            v = frame[(frame["week_day"]==week_day)&(frame["month"]==month)&(frame["hour"]==hour)&(frame["loc_id"]==loc_id)]["num_of_people"].mean()
            if v is not np.nan:
                frame.loc[i, "num_of_people"] = v
                num_nan2 +=1
            
    if day_type == 2:
        week_day = frame.loc[i, "week_day"]
        month = frame.loc[i, "month"]
        hour = frame.loc[i, "hour"]
        v = frame[(frame["week_day"]==week_day)&(frame["month"]==month)&(frame["hour"]==hour)&(frame["loc_id"]==loc_id)]["num_of_people"].mean()
        if v is not np.nan:
            frame.loc[i, "num_of_people"] = v
            num_nan2 +=1
print(frame.shape)
frame = frame.sort_values(by=["month","day","hour","loc_id"])
frame.to_csv("/home/tanyx/dataDemo/campus2018/holiday_fix_diff1_10.csv",index=False, encoding="utf-8")

(195920, 9)


In [12]:
# 单独提取假期：
holidays = []
for i in range(len(frame)):
    if check_day.check_day(frame.loc[i, "datetime"].date()) == 2:
        holidays.append(frame.loc[i,:])
holiday_frame = pd.DataFrame(holidays, columns=frame.columns).sort_values(["month", "day", "hour", "loc_id"])
# holiday_frame.to_csv("/home/tanyx/dataDemo/campus2018/holiday.csv", index=False)

21288

In [ ]:
# holiday_frame = pd.read_csv("/home/tanyx/dataDemo/campus2018/holiday.csv")
# 计算节假日权重
weight = {}
for i in [1,3,4,5,6,7,9,10,11]:
    weight[i] = {}
    for hour in range(24):
        weight[i][hour] = [1 for i in range(34)]
        for loc_id in range(1,34):
            holiday_value = holiday_frame[(holiday_frame["month"]==i)&(holiday_frame["hour"]==hour)&(holiday_frame["loc_id"]==loc_id)]["num_of_people"].mean()
            all_value = frame[(frame["month"]==i)&(frame["hour"]==hour)&(frame["loc_id"]==loc_id)]["num_of_people"].mean()
            weight[i][hour][loc_id] = holiday_value / all_value
weight_list = []
for month, value in weight.items():
    for hour, loc_ids in value.items():
        for loc_id in range(1,34):
            w = loc_ids[loc_id]
            weight_list.append([month, hour, loc_id, w])
weight_frame = pd.DataFrame(weight_list, columns=["month","hour","loc_id","weight"])
weight_frame.to_csv("/home/tanyx/dataDemo/campus2018/holiday_weight.csv", index=False)